In [ ]:
BRANCH = 'r1.11.0'

In [ ]:
"""
このノートブックはローカル（すべての依存ファイルとGPUがある場合）でも、Google Colab上でも実行可能です。

Colabのセットアップ手順は以下の通りです。
1. 新しいPython 3ノートブックを開く。
2. 2. GitHubからこのノートブックをインポートする（File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL）。
3. GPUを搭載したインスタンスに接続します（Runtime -> Change runtime type -> Select "GPU" for hardware accelerator）。
4. このセルを実行して依存関係を設定する。
"""
# Google Colabを使用していて、ローカルで実行していない場合は、次のセルを実行してください。

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]


In [ ]:
# Colabを使っていない場合、以下のエラーを回避するためにjupyter notebookをアップグレードする必要があるかもしれません。
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# このセルを実行した後、カーネルを再起動してください。

In [ ]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

# タスクの説明
自動音声認識（ASR）システムは通常、句読点や単語の大文字小文字がないテキストを生成します。
このチュートリアルでは、ASR出力をより読みやすくし、名前付きエンティティ認識、機械翻訳、音声合成モデルのパフォーマンスを高めるために、文中の各単語の句読点と大文字を予測するモデルをNeMoに実装する方法について説明します。
ここでは、事前学習済みのBERTモデルを使用して、このタスクのモデルを学習する方法を説明します。
学習データセットの各単語について、次のことを予測する。

- 単語の後に続く句読点と、その単語が大文字かどうかを予測する。

# データセット
このモデルは、以下のフォーマットに従っていれば、どのようなデータセットでも動作させることができます。
学習・評価データは、text.txtとlabels.txt*の2つのファイルに分割される。
**text.txt**ファイルの各行は、単語をスペースで区切ったテキスト列である。[WORD] [SPACE] [WORD] [SPACE] [WORD]といった具合である。



```
when is the next flight to new york
the next flight is ...
...
```



**labels.txt**ファイルには、text.txtの各単語に対応するラベルが、スペースで区切られて格納されています。labels.txtファイルの各ラベルは、2つの記号で構成されています。

- ラベルの最初の記号は、単語の後にどのような句読点を付けるかを示します（Oは句読点が不要であることを意味します）。
- 2番目の記号は、単語を大文字にするかどうかを決めます（ここで、Uは単語を大文字にすること、Oは大文字にする必要がないことを示します）。

このチュートリアルでは、カンマ、ピリオド、疑問符のみを取り上げ、その他の句読点は削除しています。もっと多くの句読点を使いたい場合は、データセットを更新して必要なラベルを追加してください。

**labels.txt**の各行は次のような形式であるべきである。
[LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (labels.txtの場合). 
例えば、上記のtext.txtファイルのラベルは次のようになります。



```
OU OO OO OO OO OO OU ?U
OU OO OO OO ...
...
```



このチュートリアルで使用したこのタスクのすべての可能なラベルの完全なリストは次のとおりです。 `OO, ,O, .O, ?O, OU, ,U, .U, ?U.`

## データをダウンロードし、前処理を行う¶。

このノートでは、[Tatoeba collection of sentences](https://tatoeba.org/eng) から英語の例文のサブセットを使用します。このスクリプトはTatoebaデータをダウンロードして前処理をします[NeMo/examples/nlp/token_classification/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_tatoeba_data.py)。なお、このモデルでさらに実験を行う場合は、NUM_SAMPLES=-1に設定し、モデルの性能を向上させるために他のデータセットも含めることを検討してください。


In [ ]:
DATA_DIR = 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED'
WORK_DIR = 'PATH_TO_A_DIRECTORY_WHERE_SCRIPTS_FOR_THIS_TUTORIAL_ARE_SAVED'
MODEL_CONFIG = "punctuation_capitalization_config.yaml"

# model parameters
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 10000

In [ ]:
## Tatoebaデータのダウンロードと前処理を行うget_tatoeba_data.pyスクリプトをダウンロードする。
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_tatoeba_data.py'):
    print('Downloading get_tatoeba_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_tatoeba_data.py', WORK_DIR)
else:
    print ('get_tatoeba_data.py is already exists')

In [ ]:
# データをダウンロードし、前処理を行う
# --clean_dir フラグは、生の Tataoeba データを削除する。
! python $WORK_DIR/get_tatoeba_data.py --data_dir $DATA_DIR --num_sample $NUM_SAMPLES --clean_dir

上記のセルを実行すると、dataフォルダに学習に必要な以下の4つのファイルが格納されます（`--cleanan_dir`を使用しない場合は、Tatoebaの生データが存在する可能性があります）。
- labels_dev.txt
- labels_train.txt
- text_dev.txt
- text_train.txt


In [ ]:
! ls -l $DATA_DIR

In [ ]:
# データを見てみよう
print('Text:')
! head -n 5 $DATA_DIR/text_train.txt

print('\nLabels:')
! head -n 5 $DATA_DIR/labels_train.txt

このように、`get_tatoeba_data.py`スクリプトは、Tatoebaをダウンロードするだけでなく、ラベルの作成も行っています。もし、自分のデータを前処理したい場合は、[examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py)スクリプトを使用してください。

In [ ]:
cwd = os.getcwd()
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py \
    --source_file $DATA_DIR/text_train.txt \
    --output_dir $DATA_DIR/my_train_preprocessed

In [ ]:
!ls $DATA_DIR/my_train_preprocessed -l

# tarred dataset

データセットが大きすぎてメモリに保存できない場合、tarred datasetを使うことができる。tarデータセットとは、モデルに渡すことのできるバッチを含むtarファイルの集合である。

すべてのtarファイルには同じ数のバッチが含まれるので、もしデータセット内のバッチの数がパラメータ `--num_batches_per_tar_file` 値で均等に割り切れない場合、最大で `--num_batches_per_tar_file - 1` 個のバッチが失われる可能性があります。

In [ ]:
# テキストファイル、ラベルファイルの行数
!wc -l $DATA_DIR/text_train.txt
!wc -l $DATA_DIR/labels_train.txt

In [ ]:
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py \
    --text $DATA_DIR/text_train.txt \
    --labels $DATA_DIR/labels_train.txt \
    --output_dir $DATA_DIR/train_tarred \
    --num_batches_per_tarfile 5 \
    --tokens_in_batch 1024 \
    --lines_per_dataset_fragment 4000 \
    --tokenizer_name bert-base-uncased \
    --n_jobs 2

In [ ]:
!ls $DATA_DIR/train_tarred -l

In [ ]:
!ls $DATA_DIR/train_tarred/*.tar | wc -l  # tarファイル数

In [ ]:
!ls $DATA_DIR/train_tarred/ | grep -v '.tar'  # すべて非tarファイル

もし、圧縮されたデータセットを使用したい場合は、以下の設定が必要である。
- 設定パラメータ `model.train_ds.tar_metadata_file` にメタデータの JSON ファイルを渡す。
- model.train_ds.use_tarred_dataset=true` に設定してください。

# モデル設定

Punctuation and Capitalization Modelでは、事前に学習した[BERT](https://arxiv.org/pdf/1810.04805.pdf)モデルの上に、2つのトークンレベルの分類器を共同で学習させています。
- 句読点を予測する分類器と、大文字を予測する分類器です。
- もう1つは大文字を予測する分類器です。

このモデルは、複数の重要なセクションを宣言した設定ファイルで定義されています。それらは以下の通りです。
- **モデル**。言語モデル、トークン分類器、オプティマイザ、スケジューラ、データセット、その他関連情報など、モデルに関連するすべての引数。

- **trainer**: PyTorch Lightningに渡す任意の引数

完全な設定の説明は[docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model)を参照してください。

In [ ]:
# モデルの設定ファイルをダウンロードする 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [ ]:
# この行は、モデルの全設定を表示します。
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

# コンフィグ内のデータのセットアップ

設定ファイルには、`common_dataset_parameters`、`train_ds`、`validation_ds` という辞書が含まれています。これらは、対応するconfigのDatasetとDataLoaderを設定するために使用される設定ファイルである。

パラメータ `train_ds.ds_item` と `validation_ds.ds_item` には、train と dev のデータセットが格納されたディレクトリを指定する。

複数のデータセットで評価したい場合は、評価用ファイルが格納されているディレクトリを以下のように指定する。

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` （パスと角括弧の間にスペースがないことに注意）。

また、`model.train_ds.ds_item`を含むいくつかの設定項目には、値の代わりに`???`

それでは、configにデータディレクトリのパスを追加してみましょう。

In [ ]:
# このチュートリアルでは、train と dev のデータは同じフォルダーにあります。
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item=DATA_DIR
del config.model.test_ds  # テストデータはなく、trainとdevのみです。

# PyTorch Lightningトレーナーの構築

NeMoのモデルは主にPyTorch Lightningのモジュールなので、PyTorch Lightningのエコシステムと完全に互換性があります。

まずはTrainerオブジェクトをインスタンス化しましょう。

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# いくつかのトレーナー設定を変更する
# GPU が利用可能かどうかを確認し、それを使用する
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# 混合精度トレーニングの場合は、precision=16、amp_level=01としてください。

# 最大エポック数を1に減らし、短時間での学習が可能
config.trainer.max_epochs = 1

# 分散学習フラグの削除
config.trainer.strategy = None

trainer = pl.Trainer(**config.trainer)

# NeMoの実験をセットアップする¶。

NeMoには実験マネージャがあり、ロギングやチェックポイントを処理してくれますので、それを使ってみましょう。

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# exp_dir は、簡単にアクセスできるように、現在の実験へのパスを提供します。
exp_dir = str(exp_dir)
exp_dir

# モデルトレーニング

モデルを初期化する前に、モデル設定のいくつかを変更したい場合があります。例えば、事前に学習したBERTモデルを変更したい場合があります。

In [ ]:
# BERT-like モデルの全対応機種一覧
print(nemo_nlp.modules.get_pretrained_lm_models_list())

PRETRAINED_BERT_MODEL = "bert-base-uncased"

In [ ]:
# 指定された上記のモデルパラメータをコンフィグに追加する
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = NUM_SAMPLES


さて、これでモデルを初期化する準備が整いました。モデル初期化呼び出しの間、データセットとデータローダーは、訓練と評価のために準備されます。
また、事前に学習された BERT モデルがダウンロードされますが、選択した BERT モデルのサイズによっては、最大で数分かかることがあります。

In [ ]:
# モデルの初期化
# この段階で、学習と評価のためのデータセットとデータローダーが準備される
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

## トレーニングの進捗をモニタリングする
オプションで、Tensorboardのビジュアライゼーションを作成し、トレーニングの進捗をモニタリングすることができます。

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# TensorBoardノートブック拡張を読み込む
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# トレーニング開始
trainer.fit(model)

# tarred データセットによる学習

In [ ]:
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = f'{DATA_DIR}/train_tarred'
config.model.train_ds.use_tarred_dataset = True
# `use_tarred_dataset=true` の場合は、メタデータファイル名のみが必要である。
config.model.train_ds.tar_metadata_file = 'metadata.punctuation_capitalization.tokens1024.max_seq_length512.bert-base-uncased.json'
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # テストデータはなく、trainとdevのみです。

# トレーナー
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.max_epochs = 1
config.trainer.strategy = None

# エクスプレス・マネージャー
config.exp_manager.explicit_log_dir = 'tarred_experiment'

config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.validation_ds.num_samples = NUM_SAMPLES

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)
trainer.fit(model)

# 事前学習済みモデルを使った推論

モデルの性能を見るために、いくつかの例で推論を実行してみましょう。

In [ ]:
print(f"Available_models: {nemo_nlp.models.PunctuationCapitalizationModel.get_available_model_names()}\n")

pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained("punctuation_en_distilbert")
# 推論に必要な行列のリストを定義する
queries = [
        'we bought four shirts and one mug from the nvidia gear store in santa clara',
        'what can i do for you today',
        'how are you',
        'how is the weather in',
    ]
inference_results = pretrained_model.add_punctuation_capitalization(queries)
print()

for query, result in zip(queries, inference_results):
    print(f'Query   : {query}')
    print(f'Combined: {result.strip()}\n')

大量のテキストに対する推論は、スクリプト [examples/nlp/token_classification/punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py) によって行うことができます。

```
python punctuate_capitalize_infer.py \
    --input_manifest <PATH/TO/INPUT/MANIFEST> \
    --output_manifest <PATH/TO/OUTPUT/MANIFEST> \
    --pretrained_name punctuation_en_bert \
    --max_seq_length 64 \
    --margin 16 \
    --step 8
```

`<PATH/TO/INPUT/MANIFEST>` は NeMo [ASR manifest](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/datasets.html) へのパスで、句読点や大文字を復元する必要があるテキストが格納されている場所です。マニフェストが `'pred_text'` キーを含んでいる場合、`'pred_text'` 要素が処理されます。そうでなければ、句読点と大文字は `'text'` 要素にリストアされます。

`<PATH/TO/OUTPUT/MANIFEST>` は、結果を保存する NeMo ASR マニフェストへのパスです。句読点や大文字小文字が復元されたテキストは
句読点と大文字が復元されたテキストは、入力マニフェストに `'pred_text'` キーが存在する場合、 `'pred_text'` 要素に保存されます。
そうでなければ、結果は `'text'` 要素に保存されます。

また、句読点や大文字小文字を復元するために、プレーンテキストとしてテキストを渡すこともできます。スクリプトのパラメータ `--input_text` と `--output_text` についてはヘルプを参照してください。
[punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py) を参照してください。

スクリプト `examples/nlp/token_classification/punctuate_capitalize_infer.py` は、任意の長さのテキストの句読点と大文字を復元することができます。長いシーケンスは、セグメントに分割されます。
各セグメントは `--max_seq_length - 2` 個のトークンに分割されます。各セグメントは `[CLS]` と `[SEP]` トークンで始まり、`[CLS]` と `[SEP]` で終わります。
各セグメントは、前のセグメントから `--step` トークンだけオフセットされています。例えば
すべての文字がトークンで、 `--max_seq_length=5`, `--step=2` の場合、テキスト `"hello"` は次のように分割されます。
セグメント `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]` に分割されることになります。

セグメントが重複している場合は、複数のセグメントに存在するトークンの予測確率を掛け合わせてから、最適な候補を選択します。

分割は、セグメントのエッジ付近でモデルのパフォーマンスを低下させる。パラメータ `--margin` を使って、セグメントの端にある `--margin` トークンに対して予測された確率を破棄することができる。例えば、全ての文字がトークンで、 `--max_seq_length=5`, `--step=1`, `--margin=1` の場合、テキスト `"hello"` はセグメント `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'e', 'l', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]` に分割されることになります。実際の予測値を計算する前に、アスタリスクでマークされたトークンの確率は削除される。`[['[CLS]', 'h', 'e', 'l'*, '[SEP]'*], ['[CLS]'*, 'e'*, 'l', 'l'*, '[SEP]'*], ['[CLS]'*, 'l'*, 'l', 'o', '[SEP]']]`.

`add_punctuation_capitalization` メソッドでは、パラメータ `max_seq_length`, `step`, `margin` を使用することができます。

以下の例のテキストはIWSLT 2019テストデータセットからのものです。


In [ ]:
inference_results = pretrained_model.add_punctuation_capitalization(
    [
        "each of these songs represents a scene a movement in some cases a sonic revolution that completely altered the "
        "course of popular music they're all also calling cards almost for those cities songs totally linked with their "
        "city's identity and might be why you probably consider them to be music cities now the magical mythical thing "
        "the thing we kind of all love about stories like these is that those cities weren't doing anything in particular "
        "to make those moments happen there's no formula for capturing lightning in a bottle a formula didn't give us "
        "grunge music or introduce tupock to dr dray and there's definitely no blueprint for how to open your record "
        "business in a south memphis neighborhood that turns out is home to booker t jones william bell and albert king "
        "so this is just something that happens and right when the stars perfectly align great music just happens and "
        "in the meantime new york and nashville can churn out the hits that come through our radios define our "
        "generations and soundtrack our weddings and our funerals and everything in between i don't know about you but "
        "the very idea of that is just deadly boring to me there are musicians all around you making powerful important "
        "music and thanks to the internet and it's limitless possibilities for creators to create music and fans to "
        "discover that music those zyite guy songs don't have to be handed down to us from some conference room full of "
        "songwriters in a corporate high rise but also and more importantly we can't decide that it's just something that "
        "happens because music is about so much more than hits those big iconic moments that change everything it's more "
        "than just entertainment for so many of us music is truly a way to navigate life a means of self expression sure "
        "but it also helps us find ourrselfel worse and figure out who we are it connects us with other people as almost "
        "nothing else can across language barriers across social and cultural and economic divides music makes us smarter "
        "and healthier and happier music is necessary what if you lived in a city that believed that that said we're not "
        "waiting for that hit song to define us we're a music city because music is necessary by seeing music as "
        "necessary a city can build two things 1st an ecosystem to support the development of professional musicians "
        "and music business and 2nd a receptive and engaged audience to sustain them and those are the two critical "
        "elements of a music city a city whose leaders recognize the importance of music for our development as "
        "individuals our connection as a community and our viability as a vibrant place to live see smart cities music "
        "cities know that thriving night lifef a creative class culture is what attracts young talented people to cities "
        "it's what brings that lightning and no we can't predict the next egg that will hatch but we can create a city "
        "that acts like an incubator to do that 1st we got to know what we've got that means identifying and quantifying "
        "our assets we need to know them backward and forward from who and what and where they are to what their impact "
        "is on the economy let's count our recording studios and our record labels our historic landmarks and our "
        "hardcore punk clubs we should count monthly free jazz nights and weekly folk jams music schools artist "
        "development instrument shops every lay than every luther music museums open once a open year round and music "
        "festivals open just one weekend year now ideally through this process we'll create an actual asset map "
        "dropping a pin for each one allowing us to see exactly what we've got and where organic momentum is already "
        "happening because it's not enough to paint in broad strokes here when it comes to specific support for music "
        "locally and a broad understanding of a music brand nationally you've got to have the receipts next we'll need "
        "to identify our challenges now it's important to knowe that for the most part this won't be just the opposite "
        "of step one we won't gain a whole lot by simply thinking about what's missing from our map instead we need to "
        "approach this more holistically there are lots of music venues on our map awesome but are they struggling do "
        "we have a venue ladder which just means can an artist starting out at a coffee house open mike see a clear path "
        "for how they'll grow from that 25 seat room to a hundred seat room and so on or are we expecting them to go from "
        "a coffee house to a colossum maybe our challenges lie in city infrastructure public transportation affordable "
        "housing maybe like in london where the number of music venues went from 400 in 2010 to 100 in 2015 we need to "
        "think about protections against gentrification the mayor of london in december of last year actually added "
        "something called the agent of change principle to the city's comprehensive plan the name says it all if a real "
        "estate developer wants to build condos next to an existing music venue the developer is the agent of change they "
        "have to take the necessary steps for noise mitigation next and this is a very big one we need leadership and we "
        "need a strategy now we know there's a lot of magic in this mix a lot of right people right place right time and "
        "that will never stop being an important element of the way music is made the way some of the best most enduring "
        "music is made but there cannot be a leadership vacuum in 2018 surriving music cities don't often happen and "
        "don't have to happen accidentally we need elected officials who recognize the power of music and elevate the "
        "voices of creatives and they're ready to put a strategy in place in music cities from berlin to paris to bogata "
        "music advisory councils ensure that musicians have a seat at the table they're volunteer councils and they work "
        "directly with a designated advocate inside of city hall or even the chamber of commerce the strongest strategies "
        "will build music community supports like this one inward while also exporting music outward they go hand in hand "
        "when we look inward we create that place that musicians want to live and when we look outward we build "
        "opportunities for them to advance their career while also driving attention back to our city and leveraging music "
        "as a talent attraction tool and here's something else that will help with that we've got to figure out who we are "
        "now when i say austin you probably think live music capital and why because in 1991 leadership in austin saw "
        "something percolating with an existing asset and they chose to own it by recognizing that momentum naming it and "
        "claiming it they inevitably caused more live music venues to open existing spaces to add live music to their "
        "repertoire and they created a swell of civic buy in around the idea which meant that it wasn't just a slogan in "
        "some tourism pamphlet was something that locals really started to believe and take pride in now generally "
        "speaking what austin created is just an assets based narrative and when we think back to step one we know that "
        "every city will not tick every box many cities won't have recording studios like memphis or a songwriter and "
        "publishing scene like nashville and that's not a deal breaker we simply have to find the momentum happening in "
        "our city what are our unique assets in comparison to no other place so if all of that sounds like something "
        "you'd like to happen where you live here are three things you can do to move the needle 1st you can use your "
        "feet your ears and your dollars show up be that receptive and engaged audience that is so necessary for a music "
        "city to thrive pay a cover charge buy a record discover new music and please take your friends two you can use "
        "your voice buy in to the assets based narrative talk about and celebrate what your city has and three you can "
        "use your vote seek out leadership that doesn't just pay lip service to your city's music but recognizes its "
        "power and is prepared to put a strategy in place to elevate it grow it and build collaboration no there really "
        "is no telling what city could be defined by a certain scene or a certain song in the next decade but as much "
        "as we absolutely cannot predict that what we absolutely can predict is what happens when we treat music as "
        "necessary and we work to build a music city and that is a place where i want to live thank you"
    ],
    max_seq_length=128,
    step=8,
    margin=16,
    batch_size=32,
)
print(inference_results[0])

## 学習スクリプト

NeMoがローカルにインストールされている場合、[nlp/token_classification/punctuation_capitalization_train_evaluate.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/punctuation_capitalization_train_evaluate.py)でモデルを学習することも可能です。

学習スクリプトを実行するには、以下を使用します。

`python punctuation_capitalization_train_evaluate.py model.train_ds.ds_item=PATH_TO_TRAIN_DATA_DIR` とします。

NUM_SAMPLES=-1に設定し、モデルの性能を向上させるために他のデータセットを含めることを検討する。

# あなたのデータでモデルを微調整する

ゼロからモデルを学習する場合、データセットはモデルの初期化時に学習用に準備されます。事前に学習した Punctuation and Capitalization モデルを使用する場合、学習前に、学習データと評価データを設定する必要があります。

In [ ]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained('punctuation_en_distilbert')

# trainとvalidationの設定 Pytorch DataLoaders
pretrained_model.update_config_after_restoring_from_checkpoint(
    train_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_train.txt',
        'labels_file': 'labels_train.txt',
        'tokens_in_batch': 1024,
    },
    validation_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_dev.txt',
        'labels_file': 'labels_dev.txt',
        'tokens_in_batch': 1024,
    },
)

# このチュートリアルでは、fast_dev_runをTrueに設定し、実際のモデル学習のために1つのトレーニングバッチと1つの検証バッチを実行します。
fast_dev_run = True
trainer = pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
pretrained_model.set_trainer(trainer)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()
trainer.fit(pretrained_model)